In [20]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn.model_selection import KFold
import warnings; warnings.simplefilter('ignore')
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression

In [17]:
df = pd.read_csv("aggregated.csv")
df.head(3)

,MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE,Unnamed: 13
0,2.0,6.0,2017-02-25,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",1000.0,0.0,156.0,937.0,NaN
1,2.0,7.0,2017-02-26,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",739.0,0.0,153.0,937.0,NaN
2,2.0,1.0,2017-02-27,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",1028.0,0.0,158.0,937.0,NaN


In [4]:
df = df.iloc[:,:-1] #Ignoring last column (Unname: 13)
# print(len(df))

In [5]:
FL_DATE = df.iloc[:,2]
UNIQUE_CARRIER = df.iloc[:,3]
ORIGIN = df.iloc[:,5]
ORIGIN_CITY_NAME = df.iloc[:,6]
DEST = df.iloc[:,7]
DEST_CITY_NAME = df.iloc[:,8]
# UNIQUE_CARRIER.head(5), ORIGIN.head(5), ORIGIN_CITY_NAME.head(5), DEST.head(5), DEST_CITY_NAME.head(5)

In [8]:
le = preprocessing.LabelEncoder()

le.fit(FL_DATE)
encoded_FL_DATE = le.transform(FL_DATE)

le.fit(UNIQUE_CARRIER)
encoded_UNIQUE_CARRIER = le.transform(UNIQUE_CARRIER)

le.fit(ORIGIN)
encoded_ORIGIN = le.transform(ORIGIN)

le.fit(ORIGIN_CITY_NAME)
encoded_ORIGIN_CITY_NAME = le.transform(ORIGIN_CITY_NAME)

le.fit(DEST)
encoded_DEST = le.transform(DEST)

le.fit(DEST_CITY_NAME)
encoded_DEST_CITY_NAME = le.transform(DEST_CITY_NAME)

df['FL_DATE'] = encoded_FL_DATE
df['UNIQUE_CARRIER'] = encoded_UNIQUE_CARRIER
df['ORIGIN'] = encoded_ORIGIN
df['ORIGIN_CITY_NAME'] = encoded_ORIGIN_CITY_NAME
df['DEST'] = encoded_DEST
df['DEST_CITY_NAME'] = encoded_DEST_CITY_NAME

In [9]:
df.head(3)

,MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME
0,2.0,6.0,330,2,28.0,188,219,98,208,1000.0,0.0,156.0
1,2.0,7.0,331,2,28.0,188,219,98,208,739.0,0.0,153.0
2,2.0,1.0,332,2,28.0,188,219,98,208,1028.0,0.0,158.0


In [10]:
# Splitting into data and labels
X = df.iloc[:,df.columns != 'ARR_DEL15']
Y = df.iloc[:,df.columns == 'ARR_DEL15']
# print(len(X), len(Y))

5129354 5129354


In [24]:
accuracylist = []

x_for_kfold = X
kfold = KFold(n_splits=10)
for train_index, test_index in kfold.split(x_for_kfold):
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = Y.values[train_index], Y.values[test_index]
    imputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
    train_x_imputed = imputer.fit_transform(X_train)
    train_y_imputed = imputer.fit_transform(y_train)
    test_x_imputed = imputer.fit_transform(X_test)
    test_y_imputed = imputer.fit_transform(y_test)
    clf = RandomForestClassifier()
    clf.fit(train_x_imputed, train_y_imputed)
    hyp = clf.predict(test_x_imputed)
    accuracylist.append(accuracy_score(test_y_imputed, hyp))
    print("Accuracy Score :",accuracy_score(test_y_imputed, hyp))
print("10 fold cross validation Accuracy Score using RandomForestClassifier :", np.mean(accuracylist))

Accuracy Score : 0.8185894536550369
Accuracy Score : 0.8392275059656565
Accuracy Score : 0.776849353525586
Accuracy Score : 0.8286784316171998
Accuracy Score : 0.8067104019027752
Accuracy Score : 0.7536354508855898
Accuracy Score : 0.7744002651408073
Accuracy Score : 0.8084104223732052
Accuracy Score : 0.8437423845126576
Accuracy Score : 0.7642820240381335
10 fold cross validation Accuracy Score using RandomForestClassifier : 0.8014525693616648


In [25]:
from sklearn.metrics import accuracy_score, confusion_matrix

print('RandomForestClassifier Confusion Matrix:\n', confusion_matrix(test_y_imputed, hyp))

RandomForestClassifier Confusion Matrix:
 [[379159  26369]
 [ 94539  12868]]


### DecisionTree Accuracy Score = 0.7766
### RF Accuracy Score = 0.8181
### LR Accuracy Score = 0.8291
### MLP Accuracy Score = 0.8291

### Running different classifiers yielded varying results, but doing a K-Fold, where K = 10; using Random Forest Classifier gives an accuracy score of 0.8018, which is the best amongst all the Classifiers. Also we checked the confusion matrix, which appears to be better for the Random Forest Classifier compared to Logistic Regression Classifier. Even though the accuracy score's for MLP & LR Classifier's are better, their confusion matrix are not as good as that of Random Forest Classifier.
### Also trying to analyze the data made me realize the data points are unbalanced, changing that and having a balanced dataset was the next step but trying that yielded an accuracy score of 0.5124, which made me realize sometimes predicting all datapoints as no delays is better. When you run the classifier for 10 times, selecting different train test sets everytime, it predicts different values, for example sometimes all the predictions are 0's but otherwise its a mixture of both, definitely not an equal mix, but around 70% no delays and 30% delays prediction.

#### All the Logistic Regression Classifier code is below for reference.

In [21]:
accuracylist = []

x_for_kfold = X
kfold = KFold(n_splits=10)
for train_index, test_index in kfold.split(x_for_kfold):
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = Y.values[train_index], Y.values[test_index]
    imputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
    train_x_imputed = imputer.fit_transform(X_train)
    train_y_imputed = imputer.fit_transform(y_train)
    test_x_imputed = imputer.fit_transform(X_test)
    test_y_imputed = imputer.fit_transform(y_test)
    clf = LogisticRegression()
    clf.fit(train_x_imputed, train_y_imputed)
    hyp = clf.predict(test_x_imputed)
    accuracylist.append(accuracy_score(test_y_imputed, hyp))
    print("Accuracy Score :",accuracy_score(test_y_imputed, hyp))
print("10 fold cross validation Accuracy Score using LogisticRegression :", np.mean(accuracylist))

Accuracy Score : 0.8478874557449663
Accuracy Score : 0.8494549027558994
Accuracy Score : 0.7897398505856481
Accuracy Score : 0.8493359795374081
Accuracy Score : 0.8215777827599988
Accuracy Score : 0.7766188698373088
Accuracy Score : 0.8003353251386628
Accuracy Score : 0.8305711249963446
Accuracy Score : 0.8676849893261329
Accuracy Score : 0.7906030978584031
10 fold cross validation Accuracy Score using LogisticRegression : 0.8223809378540773


In [22]:
# Splitting data into train data, labels and test data, labels
# split = int(len(Y) * 0.8)
# train_X = X[:split]
# train_Y = Y[:split]
# test_X = X[split:]
# test_Y = Y[split:]

# train_X, test_X, train_Y, test_Y = train_test_split(X, Y)

imputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
train_x_imputed = imputer.fit_transform(X)
train_y_imputed = imputer.fit_transform(Y)
# test_x_imputed = imputer.fit_transform(test_X)
# test_y_imputed = imputer.fit_transform(test_Y)

# print('training lengths :',len(train_x_imputed), len(train_y_imputed))
# print('testing lengths :',len(test_x_imputed), len(test_y_imputed))

scores = cross_val_score(clf,train_x_imputed,train_y_imputed,cv=10)
print("cross_val_score :", np.mean(scores))

cross_val_score : 0.8223809469967553


In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix

print('Confusion Matrix:\n', confusion_matrix(test_y_imputed, hyp))

Confusion Matrix:
 [[405528      0]
 [107407      0]]
